In [26]:
your_file = '/home/wissal/Documents/github repos cloned/PED-DQN/pursuit/results/logs/r-sc-battery_endless-h-1-m-20x20-Pp-6x3-a-ped_dqn-tu-5000-mq-4-rb-20000-mb-32-lr-0.0001-seed-1-0508130032.txt'
import tensorflow as tf
import torch
# Parse the data from the text file and extract episodes and rewards
episodes = []
rewards = []

with open(your_file, 'r') as file:
    for line in file:
        parts = line.strip().split('\t')
        if len(parts) >= 4:
            if parts[2] == 'steps':
                episodes.append(int(parts[1]))
            elif parts[2] == 'b_rwd':
                reward_values = list(map(float, parts[3:]))
                rewards.append(reward_values)

device = "GPU" if tf.config.list_physical_devices('GPU') else "CPU"
print("TensorFlow device:", device)

# Write summaries using SummaryWriter
log_dir = 'logs'  # Directory to store the event files
with tf.summary.create_file_writer(log_dir).as_default():
    for episode, reward in zip(episodes, rewards):
        tf.summary.scalar('b_rwd', reward[0], step=episode)

import matplotlib.pyplot as plt

plt.figure(facecolor='white')

plt.plot(episodes, [reward[0] for reward in rewards], label='Reward')
plt.xlabel('Episodes')
plt.ylabel('Reward (b_rwd)')
plt.title('Episodes vs Reward')
plt.legend()
plt.grid(False)
plt.show()

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f07021807f0>>
Traceback (most recent call last):
  File "/home/wissal/anaconda3/envs/my_env/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


FileNotFoundError: [Errno 2] No such file or directory: '/home/wissal/Documents/github repos cloned/PED-DQN/pursuit/results/logs/r-sc-battery_endless-h-1-m-20x20-Pp-6x3-a-ped_dqn-tu-5000-mq-4-rb-20000-mb-32-lr-0.0001-seed-1-0508130032.txt'

In [103]:
import tensorflow
print(tensorflow.__version__)
print(tf.config.list_physical_devices('GPU'))

2.11.1
[]


In [107]:
import pandas as pd
import tensorflow as tf
import numpy as np

# Define the path to your log file
log_file = '/home/wissal/Documents/github repos cloned/PED_DQN/PED-DQN 3333/pursuit/results/logs/r-sc-battery_endless-h-1-m-20x20-Pp-9x8-a-ind_dqn-tu-5000-mq-4-rb-20000-mb-32-lr-0.0001-seed-1-0519163454.txt'

# Read the data from the file
data = pd.read_csv(log_file, delimiter="\t", header=None, names=["timestamp", "episode", "metric", *range(9)])

# Group the data by episode
grouped_data = data.groupby("episode")

# Create a summary writer
log_dir = 'logs'
summary_writer = tf.summary.create_file_writer(log_dir)

# Define a function to create summary logs
def create_summary(writer, tag, value, step):
    with writer.as_default():
        tf.summary.scalar(name=tag, data=value, step=step)
        writer.flush()

# Iterate through the grouped data and log metrics to TensorBoard
for episode, group in grouped_data:
    for metric in group["metric"].unique():
        metric_data = group[group["metric"] == metric]
        #print(metric_data)
        values = metric_data.iloc[:, 3:12].values.flatten().astype(float)
        mean_value = np.mean(values)
        
        create_summary(summary_writer, metric + '/mean', mean_value, episode)

        for i, value in enumerate(values):
            create_summary(summary_writer, metric + f'/value_{i}', value, episode)

print(f"Logs written to {log_dir}. Use `tensorboard --logdir={log_dir}` to visualize.")

Logs written to logs. Use `tensorboard --logdir=logs` to visualize.
